# Lesson 08: pandas 짧게정리

최규빈  
2023-07-24

<a href="https://colab.research.google.com/github/guebin/IP2023WIN/blob/main/posts/Day1/03_Pandas/2023-01-30-Lesson8.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# ref

<https://guebin.github.io/IP2022/2022/05/04/(10주차)-5월6일.html>

# import

In [2]:
import numpy as np
import pandas as pd

# 부분 데이터 꺼내기

`-` 판다스를 왜 써야 할까?

## 기본 인덱싱

`-` 예제1: 기본인덱싱

In [3]:
a='asdf'
a[2]

In [4]:
a[-1]

`-` 예제2: 슬라이싱

In [5]:
a='asdf'
a[1:3]

In [6]:
a[-2:]

`-` 예제3: 스트라이딩

In [7]:
a='asdfg'
a[::2]

`-` 예제4: 불가능한것

In [9]:
a='asdf'
a[[1,2]] # 정수인덱스를 리스트화 시켜서 인덱싱하는 것은 불가능 

In [10]:
a='asdf'
a[[True,True,False,False]] 

## 팬시 인덱싱

`-` 예제1: 인덱스의 리스트 (혹은 ndarray)를 전달

In [17]:
a=np.arange(55,61)
a

In [20]:
a[[1,2,-1]]

In [23]:
a[np.array([1,2,-1])]

`-` 예제2: bool로 이루어진 리스트 (혹은 ndarray)를 전달

In [21]:
a=np.arange(55,61)
a

In [22]:
a[[True,True,False,False,False,False]]

In [24]:
a[np.array([True,True,False,False,False,False])]

In [27]:
a[a<58]

## 2차원자료형의 인덱싱

`-` 예제1

In [28]:
a = np.arange(4*3).reshape(4,3)
a

In [29]:
a[0:2,1]

`-` 예제2: 차원을 유지하면서 인덱싱을 하고 싶으면?

In [30]:
a = np.arange(4*3).reshape(4,3)
a[0:2,[1]]

## Hash

`-` 예제1: (key,value)

In [31]:
d={'att':65, 'rep':45, 'mid':30, 'fin':100}
d

In [33]:
d['att'] # key를 넣으면 value가 리턴

`-` 예제2: numpy와 비교

In [44]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,200)
rep = np.random.choice(np.arange(5,21)*5,200)
mid = np.random.choice(np.arange(0,21)*5,200)
fin = np.random.choice(np.arange(0,21)*5,200)
key = ['202212'+str(s) for s in np.random.choice(np.arange(300,501),200,replace=False)]
test_dic = {key[i] : {'att':att[i], 'rep':rep[i], 'mid':mid[i], 'fin':fin[i]} for i in range(200)}
test_ndarray = np.array([key,att,rep,mid,fin],dtype=np.int64).T
del(att);del(rep);del(mid);del(fin);del(key)

학번 ’202212460’에 해당하는 학생의 출석점수를 알고 싶다면?

(풀이1)

In [45]:
test_dic['202212460']['att']

(풀이2)

In [53]:
test_ndarray[test_ndarray[:,0] == 202212460, 1] ## 이게 무슨코드야 도데체!

**(풀이2)가 (풀이1)에 비하여 불편한 점** - test_ndarray의 첫칼럼은
student id 이고 두번째 칼럼은 att라는 사실을 암기하고 있어야 한다. -
student id가 아니고 만약에 학생이름을 써서 데이터를 정리한다면 모든
자료형은 문자형이 되어야 한다. - 작성한 코드의 가독성이 없다. (위치로
접근하기 때문)

`-` 요약: hash 스타일로 정보를 추출하는 것이 유용할 때가 있다. 그리고
보통 hash 스타일로 정보를 뽑는 것이 유리하다. (사실 numpy는 정보추출을
위해 개발된 자료형이 아니라 행렬 및 벡터의 수학연산을 지원하기 위해
개발된 자료형이다)

`-` 소망: 정보를 추출할때는 hash 스타일도 유용하다는 것은 이해함 $\to$
하지만 나는 가끔 넘파이스타일로 정보를 뽑고 싶은걸? 그리고 딕셔너리
형태가 아니고 엑셀처럼(행렬처럼) 데이터를 보고 싶은걸? $\to$ pandas의
개발

# pandas 개발동기

## 엑셀처럼

***엑셀처럼 데이터를 테이블 형태로 정리하고 싶다!!***

In [173]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,20)
rep = np.random.choice(np.arange(5,21)*5,20)
mid = np.random.choice(np.arange(0,21)*5,20)
fin = np.random.choice(np.arange(0,21)*5,20)
key = ['202212'+str(s) for s in np.random.choice(np.arange(300,501),20,replace=False)]
test_dic = {key[i] : {'att':att[i], 'rep':rep[i], 'mid':mid[i], 'fin':fin[i]} for i in range(20)}

In [174]:
test_dic

-   테이블형태로 보고 싶다.

(방법1) – 행렬이기는 하지만 방법 2,3,4,5 에 비하여 우리가 원하는 만큼
가독성을 주는 형태는 아님..

In [175]:
test_ndarray = np.array([key,att,rep,mid,fin],dtype=np.int64).T
test_ndarray

(방법2)

In [176]:
pd.DataFrame(test_dic).T

(방법3)

In [177]:
test_dic2 = {'att':{key[i]:att[i] for i in range(20)}, 
             'rep':{key[i]:rep[i] for i in range(20)},
             'mid':{key[i]:mid[i] for i in range(20)},
             'fin':{key[i]:fin[i] for i in range(20)}}

In [178]:
pd.DataFrame(test_dic2)

(방법4)

In [179]:
df = pd.DataFrame({'att':att, 'rep':rep, 'mid':mid, 'fin':fin},index=key)
df

(방법5)

In [180]:
df = pd.DataFrame({'att':att, 'rep':rep, 'mid':mid, 'fin':fin})
df

In [181]:
df=df.set_index([key])
df

## 딕셔너리처럼

***해싱으로 원하는 정보를 뽑으면 좋겠다!!(마치 딕셔너리처럼)***

`-` 예제1: 출석점수를 출력

In [182]:
test_dic2['att']

In [183]:
df['att']

`-` 예제2: 학번 202212380’의 출석점수 출력

In [184]:
test_dic2['att']['202212380']

In [185]:
df['att']['202212380']

## 리스트(or넘파이)처럼

***인덱싱으로 정보를 뽑는 기능도 지원을 하면 좋겠다 (마치 리스트나
넘파이처럼)***

`-` 예제1: 첫번째 학생의 기말고사 성적을 출력하고 싶다.

In [186]:
test_ndarray[0,-1]

In [199]:
df.iloc[0,-1]

-   벼락치기: df에서 iloc이라는 특수기능을 이용하면 넘파이 인덱싱처럼
    원소출력이 가능하다.

`-` 예제2: 홀수번째 학생의 점수를 뽑고 싶다.

In [188]:
test_ndarray[::2]

In [202]:
df.iloc[::2]

`-` 예제3: 맨 끝에서 3명의 점수를 출력하고 싶다.

In [113]:
test_ndarray[-3:]

In [114]:
df.iloc[-3:]

In [191]:
df[-3:]

`-` 예제4: 맨 끝에서 3명의 점수중 마지막 2개의 칼럼만 출력하고 싶다.

In [193]:
test_ndarray[-3:,-2:]

In [194]:
df.iloc[-3:,-2:]

## 다 되는거?

***궁극: 해싱과 인덱싱을 모두 지원하는 아주 우수한 자료형을 만들고
싶음***

`-` 예제1: 중간고사 점수가 20점 이상이면서 동시에 출석점수가 60점미만인
학생들의 기말고사 점수를 출력

(방법1) 데이터베이스 스타일

In [117]:
df.query("mid >= 20 and att <60")

In [118]:
df.query("mid >= 20 and att <60")['fin']

(방법2) 넘파이 스타일이라면?

In [123]:
test_ndarray

In [124]:
test_ndarray[:,3] >= 20  ## 중간고사가 20점이상

In [125]:
test_ndarray[:,1] < 60  ## 출석이 60미만

In [128]:
(test_ndarray[:,3] >= 20) & (test_ndarray[:,1] < 60)

> note: test_ndarray\[:,3\] \>= 20 & test_ndarray\[:,1\] \< 60 와 같이
> 하면 에러가 난다. 조심하자!

In [132]:
test_ndarray[(test_ndarray[:,3] >= 20) & (test_ndarray[:,1] < 60),-1] # 이게 무슨코드야

-   구현난이도 어려움, 가독성 꽝

`-` 예제2: ’중간고사점수\<기말고사점수’인 학생들의 출석점수 평균을
구하자.

In [135]:
df.query('mid<fin')['att'].mean()

# 판다스 공부 1단계

## 데이터프레임 선언

`-` 방법1: dictionary에서 만든다.

In [137]:
pd.DataFrame({'att':[30,40,50],'mid':[50,60,70]})

In [138]:
pd.DataFrame({'att':(30,40,50),'mid':(50,60,70)})

In [139]:
pd.DataFrame({'att':np.array([30,40,50]),'mid':np.array([50,60,70])})

`-` 방법: 2차원 ndarray에서 만든다.

In [140]:
np.arange(2*3).reshape(2,3)

In [141]:
pd.DataFrame(np.arange(2*3).reshape(2,3))

## 열의 이름 부여

`-` 방법1: 딕셔너리를 통하여 만들면 딕셔너리의 key가 자동으로 열의
이름이 된다.

In [142]:
pd.DataFrame({'att':np.array([30,40,50]),'mid':np.array([50,60,70])})

`-` 방법2: pd.DataFrame()의 옵션에 columns를 이용

In [144]:
pd.DataFrame(np.arange(2*3).reshape(2,3),columns=['X1','X2','X3'])

`-` 방법3: df.columns에 원하는 열이름을 덮어씀 (1)

In [157]:
df=pd.DataFrame(np.arange(2*3).reshape(2,3))
df

In [158]:
df.columns = ['X1','X2','X3']

In [159]:
df

In [162]:
df.columns

`-` 방법4: df.columns에 원하는 열이름을 덮어씀 (2)

In [166]:
df=pd.DataFrame(np.arange(2*3).reshape(2,3))
df

In [167]:
df.columns = pd.Index(['X1','X2','X3'])

In [168]:
df

방법4가 방법3의 방식보다 컴퓨터가 이해하기 좋다. (= 불필요한 에러를
방지할 수 있다)

In [170]:
df.columns, type(df.columns)

In [171]:
['X1','X2','X3'], type(['X1','X2','X3'])

In [172]:
pd.Index(['X1','X2','X3']), type(pd.Index(['X1','X2','X3']))

## 행의 이름 부여

`-` 방법1: 중첩 dict이면 nested dic의 key가 알아서 행의 이름으로 된다.

In [206]:
pd.DataFrame({'att':{'guebin':30, 'iu':40, 'hynn':50} , 'mid':{'guebin':5, 'iu':45, 'hynn':90}})

`-` 방법2: pd.DataFrame()의 index 옵션 이용

In [212]:
pd.DataFrame({'att':[30,40,50] , 'mid':[5,45,90]}, index=['guebin','iu','hynn'])

`-` 방법3: df.index에 덮어씌움

In [223]:
df=pd.DataFrame({'att':[30,40,50] , 'mid':[5,45,90]})
df

In [226]:
df.index = pd.Index(['guebin','iu','hynn'])
#df.index = ['guebin','iu','hynn'] <- 이것도 실행 되기는 된다.
df

`-` 방법4: df.set_index() 를 이용하여 덮어씌운다

In [227]:
df=pd.DataFrame({'att':[30,40,50] , 'mid':[5,45,90]})
df

In [229]:
df.set_index(pd.Index(['guebin','iu','hynn']))

(주의) 아래는 에러가 난다.

In [230]:
df.set_index(['guebin','iu','hynn'])

In [232]:
df.set_index([['guebin','iu','hynn']]) # 꺽쇠를 한번 더 넣어주면 에러를 피할수 있다. 

## 자료형, len, shape, for문의 반복변수

In [233]:
df = pd.DataFrame({'att':[30,40,50],'mid':[5,45,90]})
df

`-` type

In [234]:
type(df)

`-` len

In [237]:
len(df) # row의 갯수 

`-` shape

In [239]:
df.shape 

`-` for문의 반복변수

In [242]:
for k in df:
    print(k) # 딕셔너리같죠

att
mid

In [243]:
for k in {'att':[30,40,50],'mid':[5,45,90]}: 
    print(k)

att
mid

## pd.Series

`-` 2차원 ndarray가 pd.DataFrame에 대응한다면 1차원 ndarray는
pd.Series에 대응한다.

In [246]:
a=pd.Series(np.random.randn(10))
a

In [247]:
type(a)

In [248]:
len(a)

In [249]:
a.shape

In [259]:
for value in a: 
    print(value)

0.45375797778887145
-0.7162702753778524
-0.1677503294060748
0.08954572073306402
0.8740060519863648
0.40162700372357163
0.20491068603376858
1.0427224662413124
-0.3523562215445016
-1.2416580554177619

# 판다스 공부 2단계

`-` 데이터

In [261]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,20)
rep = np.random.choice(np.arange(5,21)*5,20)
mid = np.random.choice(np.arange(0,21)*5,20)
fin = np.random.choice(np.arange(0,21)*5,20)
key = ['202212'+str(s) for s in np.random.choice(np.arange(300,501),20,replace=False)]

In [264]:
df=pd.DataFrame({'att':att,'rep':rep,'mid':mid,'fin':fin},index=key)
df

## 첫번째 칼럼을 선택

`-` 방법1

In [290]:
df.att

`-` 방법2: dict스타일

In [291]:
df['att']

`-` 방법3: dict스타일

In [292]:
df[['att']]

-   df.att 나 df\[‘att’\]는 series를 리턴하고 df\[\[‘att’\]\]는
    dataframe을 리턴한다.

`-` 방법4: ndarray스타일

In [283]:
df.iloc[:,0] 

`-` 방법5: ndarray스타일

In [284]:
df.iloc[:,[0]]

-   df.iloc\[:,0\]은 series를 리턴하고 df.iloc\[:,\[0\]\]은 dataframe을
    리턴한다.

`-` 방법6: ndarray 스타일과 dict 스타일의 혼합

In [279]:
df.loc[:,'att'] 

`-` 방법7: ndarray 스타일과 dict 스타일의 혼합

In [280]:
df.loc[:,['att']] 

-   df.loc\[:,‘att’\]은 series를 리턴하고 df.loc\[:,\[‘att’\]\]은
    dataframe을 리턴한다.

`-` 방법7: nparray 스타일 + bool 인덱싱

In [286]:
df.iloc[:,[True,False,False,False]]

`-` 방법8: ndarray와 dict의 홉합형 + bool 인덱싱

In [287]:
df.loc[:,[True,False,False,False]]

## 여러개의 칼럼을 선택

`-` 방법1: dict 스타일

In [296]:
df[['att','fin']]

`-` 방법2: ndarray 스타일 (정수리스트로 인덱싱, 슬라이싱, 스트라이딩)

In [299]:
df.iloc[:,[0,1]] # 정수의 리스트를 전달하여 컬럼추출

In [331]:
df.iloc[:,range(2)] 

In [321]:
df.iloc[:,:2]  # 슬라이싱 , 0,1,2에서 마지막 2는 제외되고 0,1에 해당하는 것만 추출

In [305]:
df.iloc[:,::2]  # 스트라이딩

`-` 방법3: ndarray 와 dict의 혼합형

In [310]:
df.loc[:,['att','mid']] 

In [322]:
df.loc[:,'att':'mid']  # 마지막의 mid도 포함된다. 

In [320]:
df.loc[:,'rep':] 

`-` 방법4: bool을 이용한 인덱싱

In [323]:
df.iloc[:,[True,False,True,False]]

In [324]:
df.loc[:,[True,False,True,False]]

## 첫번째 행을 선택

`-` 방법1

In [338]:
df.iloc[0]

`-` 방법2

In [339]:
df.iloc[[0]]

`-` 방법3

In [340]:
df.iloc[0,:]

`-` 방법4

In [341]:
df.iloc[[0],:]

`-` 방법5

In [342]:
df.loc['202212380']

`-` 방법6

In [343]:
df.loc[['202212380']]

`-` 방법7

In [344]:
df.loc['202212380',:]

`-` 방법8

In [345]:
df.loc[['202212380'],:]

`-` 방법9

In [346]:
len(df)

In [348]:
_lst = [True]+[False]*19

In [351]:
df.iloc[_lst] 

In [352]:
df.iloc[_lst,:] 

In [353]:
df.loc[_lst] 

In [354]:
df.loc[_lst,:] 

## 여러개의 행을 선택

`-` 방법1

In [358]:
df.iloc[[0,2]] 

In [359]:
df.iloc[[0,2],:] 

`-` 방법2

In [361]:
df.loc[['202212380','202212363']] 

In [360]:
df.loc[['202212380','202212363'],:] 

`-` 그 밖의 방법들

In [365]:
df.iloc[::3] # 스트라이딩

In [366]:
df.iloc[:5]

In [371]:
df.loc[:'202212312']

In [377]:
df.loc[list(df.att<80),'rep':]

In [375]:
df.loc[df.att<80,'rep':]

In [379]:
df.iloc[list(df.att<80),1:]

`-` 아래는 에러가 난다 주의!

In [383]:
df.iloc[df.att<80,1:]

## query ($\star$)

`-` 예제1

In [385]:
df.query('att==90 and mid>30')

`-` 예제2

In [386]:
df.query('att<rep and mid<fin')

`-` 예제3

In [387]:
df.query('att < rep < 80')

`-` 예제4

In [388]:
df.query('50 < att <= 90 and mid < fin')

`-` 예제5

In [389]:
df.query(' (mid+fin)/2 >=60')

`-` 예제6

In [393]:
_mean  = df.att.mean()
_mean 

In [394]:
df.query('att >= 73')

In [397]:
df.query('att >= @_mean')
# df.query('att>= _mean')은 실행 안된다.

`-` 예제7

In [416]:
df

In [419]:
df.query("index <= '202212354' or index=='202212387'")

사실 이 기능은 시계열자료에서 꽃핀다.

`-` 예제8

In [428]:
pd.date_range('20211226',periods=10)

In [430]:
_df=pd.DataFrame(np.random.normal(size=(10,4)),columns=list('ABCD'),index=pd.date_range('20211226',periods=10))
_df

In [435]:
_df.query(" '2021-12-29' <=  index <= '2022-01-03' ")

In [436]:
_df.query(" '2021-12-29' <=  index <= '2022-01-03' and A+B<C  ")

`-` query가 만능은 아니다.

In [439]:
df.columns = pd.Index(['att score', 'rep score', 'mid score', 'fin score'])

In [455]:
df.query(" att score < 90 ")
# df.loc[df['att score'] < 90,:] <-- 이렇게 구현하면 된다! (아니면 변수이름을 바꿔주는 코드를 짜든가)

# 판다스 공부 3단계

## 전치

In [457]:
ndarray = np.arange(2*3).reshape(2,3)
df = pd.DataFrame(ndarray)
df

In [458]:
ndarray.T

In [459]:
df.T

## 합

In [461]:
ndarray.sum(axis=0)

In [462]:
df.sum(axis=0)

In [463]:
ndarray.sum(axis=1)

In [464]:
df.sum(axis=1)

## cumsum

In [465]:
df

In [466]:
ndarray.cumsum(axis=0)

In [467]:
df.cumsum(axis=0)

In [468]:
ndarray.cumsum(axis=1)

In [469]:
df.cumsum(axis=1)

## 형태변환

In [470]:
ndarray.tolist()

In [478]:
df.to_dict()

In [477]:
df.to_numpy()

In [476]:
df.to_numpy().tolist()

# Quiz

`-` 아래의 데이터프레임에서 1,3번째 열을 추출하라.

In [481]:
df= pd.DataFrame({'att':[90,90,95],'rep':[80,90,90],'mid':[50,60,70], 'fin':[70,80,50]})
df